<a href="https://colab.research.google.com/github/GeorgeSherif/NLP-ChatEGP/blob/main/paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
!pip install transformers sentencepiece sacremoses
from transformers import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
path = "/content/gdrive/My Drive/NLP/all-data.csv"

In [39]:
df = pd.read_csv(path,encoding = "ISO-8859-1")

In [40]:
df.head

<bound method NDFrame.head of          label                                           sentence
0      neutral  According to Gran , the company has no plans t...
1      neutral  Technopolis plans to develop in stages an area...
2     negative  The international electronic industry company ...
3     positive  With the new production plant the company woul...
4     positive  According to the company 's updated strategy f...
...        ...                                                ...
4841  negative  LONDON MarketWatch -- Share prices ended lower...
4842   neutral  Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843  negative  Operating profit fell to EUR 35.4 mn from EUR ...
4844  negative  Net sales of the Paper segment decreased to EU...
4845  negative  Sales in Finland decreased by 10.5 % in Januar...

[4846 rows x 2 columns]>

In [41]:
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--tuner007--pegasus_paraphrase/snapshots/0159e2949ca73657a2f1329898f51b7bb53b9ab2/config.json
Model config PegasusConfig {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_

In [42]:
def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences, num_beams):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [43]:
headerList = ['label', 'sentence']
  
# converting data frame to csv
df.to_csv(path, header=headerList, index=False)

file2 = pd.read_csv(path)

In [44]:
file2['sentence'][1]

'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .'

In [45]:
df1 = pd.DataFrame(index=range(604), columns=['label' , 'sentence'])

In [47]:
j = 0
for i in range (len(file2)):
  if(file2['label'][i]=='negative'):
    df1['label'][j] = 'negative'
    df1['sentence'][j] = get_paraphrased_sentences(model, tokenizer, file2['sentence'][i], num_beams=1, num_return_sequences=1)[0]
    print(df1['sentence'][j])
    j = j + 1
    print(j)

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 60,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.29.2"
}



The operating profit fell from the previous year, with a gain of 12.3 million dollars.
602


Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 60,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.29.2"
}



Net sales of the Paper segment decreased to 221.6 million dollars in the second quarter of 2009, from 241.1 million dollars in the second quarter of 2008, while operating profit rose to 8.0 million dollars from 7.6 million dollars.
603
In January, sales in Finland decreased by 10% while sales outside of the country decreased by 17 %.
604


In [48]:
file2 = file2.append(df1)
file2

<ipython-input-48-7650fbab7efa>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file2 = file2.append(df1)


,label,sentence
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
599,negative,The cargo handling group posted a surprise dro...
600,negative,The share prices in London ended lower on Mond...
601,negative,The operating profit fell from the previous ye...
602,negative,Net sales of the Paper segment decreased to 22...


In [49]:
file2 = file2.rename(columns={"label":"Sentiment", "sentence": "Sentence"})


In [50]:
print((file2['Sentiment'] == 'negative').sum())

1208


In [51]:

file_path = '/content/gdrive/MyDrive/NLP/Augmented.csv'
file2.to_csv(file_path, index=False)
     